In [8]:
import pandas as pd
import numpy as np

In [9]:
# !mkdir -p "/content/drive/MyDrive/PRIVILEGE_ESCALATION_ATTACK_DETECTION/dataset/"

In [10]:
# !tar -xvjf "/content/drive/MyDrive/PRIVILEGE_ESCALATION_ATTACK_DETECTION/CERT_r4.2/r4.2.tar.bz2" -C "drive/MyDrive/PRIVILEGE_ESCALATION_ATTACK_DETECTION/dataset/"

In [11]:
# !tar -xvjf "/content/drive/MyDrive/PRIVILEGE_ESCALATION_ATTACK_DETECTION/ANSWERS/answers.tar.bz2" -C "drive/MyDrive/PRIVILEGE_ESCALATION_ATTACK_DETECTION/dataset/"

In [12]:
BASE_DIR = "/content/drive/MyDrive/PRIVILEGE_ESCALATION_ATTACK_DETECTION/dataset"

In [13]:
import os
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier
from sklearn.metrics import classification_report, confusion_matrix
import warnings
warnings.filterwarnings("ignore")

BASE_DIR = "/content/drive/MyDrive/PRIVILEGE_ESCALATION_ATTACK_DETECTION/dataset"
print("Ready")

Ready


In [14]:
def load_scenario_3_answers():
    rows = []
    scenario_dir = "/content/drive/MyDrive/PRIVILEGE_ESCALATION_ATTACK_DETECTION/dataset/answers/r4.2-3"

    if not os.path.exists(scenario_dir):
        print(f"ERROR: Directory not found: {scenario_dir}")
        print("Available folders in dataset/:")
        print(os.listdir("/content/drive/MyDrive/PRIVILEGE_ESCALATION_ATTACK_DETECTION/dataset"))
        return None

    print(f"Loading malicious events from: {scenario_dir}")
    for file in os.listdir(scenario_dir):
        if file.endswith(".csv"):
            path = os.path.join(scenario_dir, file)
            print(f"  * {file}")
            with open(path, "r") as f:
                for line in f:
                    line = line.strip()
                    if not line:
                        continue
                    parts = [p.strip() for p in line.split(",", 5)]
                    if len(parts) < 6:
                        continue
                    activity, id_, datetime_str, user, pc, description = parts
                    try:
                        timestamp = pd.to_datetime(datetime_str, format="%m/%d/%Y %H:%M:%S")
                        rows.append({"user": user, "timestamp": timestamp})
                    except:
                        continue
    df = pd.DataFrame(rows)
    print(f"Loaded {len(df)} malicious events from Scenario 3")
    return df

malicious_events = load_scenario_3_answers()
if malicious_events is None:
    raise SystemExit("Stop here")

# Create one interval per malicious user
malicious_intervals = (
    malicious_events.groupby("user")["timestamp"]
    .agg(["min", "max"])
    .rename(columns={"min": "start", "max": "end"})
    .reset_index()
)
malicious_users = set(malicious_intervals["user"])
print(f"Malicious users ({len(malicious_users)}): {sorted(malicious_users)}")

Loading malicious events from: /content/drive/MyDrive/PRIVILEGE_ESCALATION_ATTACK_DETECTION/dataset/answers/r4.2-3
  * r4.2-3-BBS0039.csv
  * r4.2-3-BSS0369.csv
  * r4.2-3-CSC0217.csv
  * r4.2-3-GTD0219.csv
  * r4.2-3-CCA0046.csv
  * r4.2-3-JLM0364.csv
  * r4.2-3-JGT0221.csv
  * r4.2-3-MPM0220.csv
  * r4.2-3-MSO0222.csv
  * r4.2-3-JTM0223.csv
Loaded 213 malicious events from Scenario 3
Malicious users (12): ['BBS0039', 'BSS0369', 'CCA0046', 'CSC0217', 'FAW0032', 'FBA0348', 'GTD0219', 'JGT0221', 'JLM0364', 'JTM0223', 'MPM0220', 'MSO0222']


In [15]:
relevant_users = malicious_users | set(
    np.random.choice(
        list(set(pd.read_csv(os.path.join(BASE_DIR, "r4.2/logon.csv"), usecols=["user"])["user"]) |
             set(pd.read_csv(os.path.join(BASE_DIR, "r4.2/device.csv"), usecols=["user"])["user"]) |
             set(pd.read_csv(os.path.join(BASE_DIR, "r4.2/email.csv"), usecols=["user"])["user"])),
        # size=len(malicious_users),
        size=25,
        replace=False
    )
)

print(f"Total users kept: {len(relevant_users)} (all malicious + equal benign)")

def load_filtered(file_name):
    path = os.path.join(BASE_DIR, "r4.2", file_name)
    df = pd.read_csv(path, low_memory=False)
    df = df[df["user"].isin(relevant_users)].copy()
    df["timestamp"] = pd.to_datetime(df["date"], format="%m/%d/%Y %H:%M:%S")
    df = df.set_index("timestamp").sort_index()
    print(f"{file_name}: {len(df):,} rows")
    return df

logon_df  = load_filtered("logon.csv")
device_df = load_filtered("device.csv")
email_df  = load_filtered("email.csv")

Total users kept: 37 (all malicious + equal benign)
logon.csv: 39,031 rows
device.csv: 36,234 rows
email.csv: 92,491 rows


In [16]:
df = pd.concat([logon_df, device_df, email_df]).sort_index()
print(df.columns)
print(f"\nMerged dataset: {len(df):,} rows")

Index(['id', 'date', 'user', 'pc', 'activity', 'to', 'cc', 'bcc', 'from',
       'size', 'attachments', 'content'],
      dtype='object')

Merged dataset: 167,756 rows


In [17]:
df["label"] = 0
for _, row in malicious_intervals.iterrows():
    mask = (df["user"] == row["user"]) & \
           (df.index >= row["start"]) & \
           (df.index <= row["end"])
    df.loc[mask, "label"] = 1

print(f"Malicious events labeled: {df['label'].sum():,}")
print(f"Class distribution → Benign: {len(df)-df['label'].sum():,} | Malicious: {df['label'].sum():,}")

Malicious events labeled: 4,773
Class distribution → Benign: 162,983 | Malicious: 4,773


In [18]:
# Step 1: Fix the "source" column (email / device / logon / http)
def get_source(row):
    if 'to' in row and pd.notna(row['to']):
        return 'email'
    if 'url' in row and pd.notna(row['url']):
        return 'http'
    if row.name in ['Connect', 'Disconnect']:
        return 'device'
    if row.name in ['Logon', 'Logoff']:
        return 'logon'
    return 'other'

df["source"] = df.apply(get_source, axis=1)

# Step 2: Define features per user per day
def add_features(group):
    f = {}
    f["logon_count"]      = (group["activity"] == "Logon").sum()
    f["logoff_count"]     = (group["activity"] == "Logoff").sum()
    f["connect_count"]    = (group["activity"] == "Connect").sum()
    f["disconnect_count"] = (group["activity"] == "Disconnect").sum()
    f["email_count"]      = (group["source"] == "email").sum()
    f["http_count"]       = (group["source"] == "http").sum()
    f["after_hours"]      = ((group.index.hour < 7) | (group.index.hour > 19)).sum()
    f["weekend"]          = (group.index.weekday >= 5).sum()
    f["total_events"]     = len(group)
    f["label"]            = group["label"].any()  # malicious day = 1
    return pd.Series(f)

# Step 3: Group by user and date -> daily features
daily_features = (
    df.groupby(["user", df.index.date])
      .apply(add_features)
      .reset_index()
      .rename(columns={0: "date"})
)

# Fix column name properly
daily_features = daily_features.rename(columns={"level_1": "date"})

print(f"Daily features created: {daily_features.shape}")
print("Columns:", daily_features.columns.tolist())
daily_features.head()

Daily features created: (10927, 12)
Columns: ['user', 'date', 'logon_count', 'logoff_count', 'connect_count', 'disconnect_count', 'email_count', 'http_count', 'after_hours', 'weekend', 'total_events', 'label']


,user,date,logon_count,logoff_count,connect_count,disconnect_count,email_count,http_count,after_hours,weekend,total_events,label
0,AIB0948,2010-01-04,2,1,9,9,1,0,0,0,22,False
1,AIB0948,2010-01-05,2,1,9,9,1,0,0,0,22,False
2,AIB0948,2010-01-06,2,2,9,9,1,0,0,0,23,False
3,AIB0948,2010-01-07,2,1,9,9,1,0,0,0,22,False
4,AIB0948,2010-01-08,2,1,9,9,1,0,0,0,22,False


In [19]:
# Sort by date
daily_features = daily_features.sort_values("date").reset_index(drop=True)

# Features and target
X = daily_features.drop(columns=["user", "date", "label"])
y = daily_features["label"]

# Temporal split (70% earlier days -> train, 30% later days -> test)
split_date = daily_features["date"].quantile(0.7)   # this is a pandas Timestamp
split_date = pd.Timestamp(split_date).date()       # convert to plain date

train_mask = daily_features["date"] <= split_date

X_train, X_test = X[train_mask], X[~train_mask]
y_train, y_test = y[train_mask], y[~train_mask]

print(f"Training period  : up to {split_date}")
print(f"Test period      : from {split_date + pd.Timedelta(days=1)} onward")
print(f"Train samples    : {len(X_train):,}")
print(f"Test samples     : {len(X_test):,}")
print(f"Malicious in test: {y_test.sum()}")

# Stacking ensemble
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

# Fix scale_pos_weight safely
neg_count = (y_train == 0).sum()
pos_count = (y_train == 1).sum()
scale_pos_weight = neg_count / pos_count if pos_count > 0 else 1

base_models = [
    ("rf",  RandomForestClassifier(n_estimators=400, class_weight="balanced", n_jobs=-1, random_state=42)),
    ("xgb", XGBClassifier(n_estimators=400, max_depth=7, scale_pos_weight=scale_pos_weight,
                          eval_metric="logloss", n_jobs=-1, random_state=42))
]

stack = StackingClassifier(
    estimators=base_models,
    final_estimator=LogisticRegression(class_weight="balanced"),
    cv=5,
    n_jobs=-1,
    passthrough=True
)

print("\nTraining stacking model...")
stack.fit(X_train, y_train)

y_pred = stack.predict(X_test)

print("\n" + "="*60)
print("FINAL RESULTS - PRIVILEGE ESCALATION DETECTION (Scenario 3)")
print("="*60)
print(classification_report(y_test, y_pred, digits=4))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("="*60)

Training period  : up to 2010-11-09
Test period      : from 2010-11-10 onward
Train samples    : 7,657
Test samples     : 3,270
Malicious in test: 142

Training stacking model...

FINAL RESULTS - PRIVILEGE ESCALATION DETECTION (Scenario 3)
              precision    recall  f1-score   support

       False     0.9973    0.9463    0.9711      3128
        True     0.4437    0.9437    0.6036       142

    accuracy                         0.9462      3270
   macro avg     0.7205    0.9450    0.7874      3270
weighted avg     0.9733    0.9462    0.9552      3270

Confusion Matrix:
[[2960  168]
 [   8  134]]


In [23]:
df.columns

Index(['id', 'date', 'user', 'pc', 'activity', 'to', 'cc', 'bcc', 'from',
       'size', 'attachments', 'content', 'label', 'source'],
      dtype='object')

In [29]:
daily_features.columns

Index(['user', 'date', 'logon_count', 'logoff_count', 'connect_count',
       'disconnect_count', 'email_count', 'http_count', 'after_hours',
       'weekend', 'total_events', 'label'],
      dtype='object')